In [1]:
%matplotlib qt
# Linear regression


2024-02-20 09:42:33.334 Python[56488:1205838] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [2]:
import matplotlib as mpl
import happi
import numpy as np
import matplotlib.pyplot as plt

jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
jet_vals[8] = [0.0, 0, 0.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

plt.rc('font', family='sans-serif', serif='Arial')
plt.rc('text', usetex=False)
plt.rc('xtick', labelsize=13)
plt.rc('ytick', labelsize=13)
plt.rc('axes', labelsize=13)
plt.rc('legend', fontsize=10)

/var/folders/2t/97rc3fl92tg15k2l_4sk5hsh0000gn/T/ipykernel_56488/1333008397.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd


In [3]:
wkdir = [
    # '/Users/yao/Desktop/Data/DS_t0/', # test 0 -> needs longer plasma and time
    # '/Users/yao/Desktop/Data/DS_t1/', # test 1 -> longer plasma and time
    # '/Users/yao/Desktop/Data/SS_t2/', # test 2 -> single shock with v=3000 km/s
    # '/Users/yao/Desktop/Data/SS_t3/', # test 3 -> single shock with v=3000 km/s
    # '/Users/yao/Desktop/Data/SS_velocities/SS_t5/',   # test 4 -> larger box size
    '/Users/yao/Desktop/Data/SS_t3000_Nia/',
    # '/Users/yao/Nextcloud/PROJECTS/Vulcan/RAL_August2023/Archive/'
    ]

In [4]:
S = happi.Open(wkdir[0], reference_angular_frequency_SI = 56375055300167.87)
# S1 = happi.Open(wkdir[1], reference_angular_frequency_SI = 56375055300167.87)


Loaded simulation '/Users/yao/Desktop/Data/SS_t3000_Nia/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Scanning for new particle diagnostics


In [5]:
# prepare constants, units

me = 9.1e-31
mp = 1836.*me
qe = 1.6e-19
ep = 8.9e-12  # epsilon_0
c  = 3.0e8
wr = S.namelist.w_r
de = c / wr
Lx = S.namelist.L_x.real * de * 1e3      # in mm
dx = S.namelist.d_x * de * 1e3           # in mm

Te = S.namelist.T_e * 511.e3             # in eV
ne = 1.0e18                              # in cm-3
ld = 7.43e2 * Te**0.5 * ne**(-0.5) * 10. # in mm
dt = S.namelist.d_t

B0 = S.namelist.B_z * (me * wr / qe)
wc = qe * B0 / me

In [6]:
B0 

9.939626937610848

In [6]:
# Function for spectrum

def spect_load_1d(wkdir,time,species,xmin,xmax):
    """
    Inputs ----
    wkdir      : string, data directory
    time       : float,  [0,1] of the time_end 
    species    : int,    which species you want to plot
                         according to your own diagnostic in the Namelist
    xmin, xmax : float,  spatial selections, NOTE: in unit of de
    
    Outputs ---
    ekin       : float,  x-axis, energy(E) in MeV
    num        : float,  y-axis, E \frac{dN}{dE} \frac{1}{N}
                                 N  is the total number of particles
                                 dN is the number of particles in each energy range (dE)
                                 dE is the energy range
    simu_time  : float,  simulation time
    """
#     S = happi.Open(wkdir)                        # No need to open the data again.
    dt = S.namelist.d_t                            # get simulation timestep
    ts = S.ParticleBinning(species).getTimesteps() # get all the timesteps available for this diag.
    iteration = int(ts[-2] * time)                 # choose the time you want by a factor of [0,1]*T_end
    simu_time = iteration * dt / wr                # convert it into real unit
    pb = S.ParticleBinning(species).get(iteration) # get the diag. data from the chosen timestep
    
    xx   = pb['x'] * de * 1e3                      # get position information and convert the unit to mm
    ekin = pb['ekin'] * 0.511                      # get energy information (E) and convert the unit to MeV -- the x-axis
    data = pb['data'][0]                           # get the particle number information
    data_selected = data[(xx>xmin) & (xx<xmax),:]  # select particle number according to position
                                                   # for our case now, this doesn't make much difference.
    
    data_ekin = np.sum(data_selected,axis=0)       # dN, ?? can't remember... you might want to check what it does
    dE = ekin[1]-ekin[0]                           # dE, energy range
    total_weight = np.sum(data)                    # N, total number of the selected portion
    num = data_ekin * ekin / total_weight / dE     # E \frac{dN}{dE} \frac{1}{N} -- the y-axis
    
    return ekin, num, simu_time

# Function for 2D streak map

def prepare_streak_2D(case, num, field_name, frame):
    
    """
    case: string, Smilei case
    num: '0', instant; '1', averaged 
    field_name: string, name of the field
    frame: float, velocity of the reference frame
    """
    var = case.Field(num,field_name,units=["mm","ns","cm^-3","T"]) # find the data
    data_var = var.getData()                           # get the data out
    data_var_2D = np.vstack(data_var)                  # transform it into 2D with (t,x)
    data_var_t = var.getTimes()                        # get the t-axis
    data_var_x = var.getAxis('x')                      # get the x-axis
    
    data_var_x_2D, data_var_t_2D = np.meshgrid(data_var_x, data_var_t, indexing='xy') # make the interpolation
    
    data_var_x_2D_frame = data_var_x_2D - frame * data_var_t_2D - Lx/2.               # change the reference frame (and the label) 
                                                                                      # note the unit used
    return data_var_x_2D, data_var_x_2D_frame, data_var_t_2D, data_var_2D

def Particle_trajectory(case, species, mass, condition):
    
    """
    case: string, Smilei case
    species: string, name of the traced particle
    mass: float, mass of the traced particle, used when calcuating momentum/energy etc.
    condition: string, Smilei formatted, to select particles satisfying a certain conditions.
    frame: float, velocity of the reference frame
    """
    
#     me = 9.1e-31
#     c  = 3.0e8
#     wr = S.namelist.w_r
#     de = c / wr 
    
    tp = case.TrackParticles(species,
                             select = condition,
                             axes=['x','px','py','pz','Ex','Ey','Bz','w'],
                             units=[
                                 # 'mm',
                                 'ns'
                             ],                     # TODO: Use normalized unit during calculation
                             sort=True,
                            )
    data = tp.getData()                                           # get data
    time_tmp = data['times']                                      # get time, in unit of wpe^{-1}
    time_ns = np.linspace(0,tp.getTimes()[-1],time_tmp.size)      # transform time array into unit of ns.
    
    tp_px = data['px']                                            # extract all kinds of data
    tp_py = data['py']
    tp_pz = data['pz']
    tp_Ex = data['Ex']
    tp_Ey = data['Ey']
    tp_Bz = data['Bz']
    tp_x  = data['x']
    tp_weight = data['w']
    
#     tp_x_frame = tp_x * de * 1e3 - frame * time_ns              # change reference frame, do it later.
    
    tp_E  = (tp_px**2 + tp_py**2 + tp_pz**2)*(me*c)**2 / 2. / mass / qe / 1e6  # energy, in Unit of MeV
    tp_E = np.where(np.isfinite(tp_E),tp_E,0.)                                 # replace the inf/nan with 0.
    
    # find the particle with the highest energy at the last timestep 
    # (end of the simulation)
    [timestep, particle_number] = np.unravel_index(tp_E[-1].argmax(), tp_E.shape)
    
    tp_x_sub = tp_x[:,particle_number]
    tp_E_sub = tp_E[:,particle_number]
    
    return tp_x, tp_E, tp_px, tp_py, tp_pz, time_ns, tp_x_sub, tp_E_sub, tp_Ex, tp_Ey, tp_Bz, tp_weight, timestep, particle_number

## Calculate the kinetic energy gained by Ex, Ey, and the total.

def cal_kinetic_ene(part_id, E, Ex, Ey, px, py, time_ns):
    '''
    calculate the kinetic energy gained by Ex, Ey, and the total.
    '''
#     P_Ex = qe*ion2t_Ex[:, part_id]*kE*ion2t_px[:,part_id]*kp/mp
#     P_Ey = qe*ion2t_Ey[:, part_id]*kE*ion2t_py[:,part_id]*kp/mp
    
    P_Ex = qe*Ex[:, part_id]*kE*px[:,part_id]*me*c/mp
    P_Ey = qe*Ey[:, part_id]*kE*py[:,part_id]*me*c/mp

    dt = time_ns[2] - time_ns[1]
    
    t_P_tot5 = []
    for i in range(len(time_ns)-1):
        t_P_tot5.append((time_ns[i+1]+time_ns[i])/2.)

    K_Ex_5 = []
    for i in range(len(Ex[:, part_id])-1):
        K_Ex_5.append(sum(dt*1e-9*np.array(P_Ex[0:i+1]))/conv_MeV_J*1e3)

    K_Ey_5 = []
    for i in range(len(Ex[:, part_id])-1):
        K_Ey_5.append(sum(dt*1e-9*np.array(P_Ey[0:i+1]))/conv_MeV_J*1e3)


    # average functions every 10 timesteps
    def average(func):
        av = []
        for i in np.arange(0, len(func)-1,10):
            av.append((func[i]+func[i+1]+func[i+2]+func[i+3]+func[i+4]+func[i+5]+func[i+6]+func[i+7]+func[i+8]+func[i+9])/10)
        return av
    
    def average_3(func):
        av_3 = []
        for i in np.arange(0, len(func)-1,3):
            av_3.append((func[i]+func[i+1]+func[i+2])/3)
        return av_3

    K_tot = np.array(average_3(E[:, part_id]*1e3))
    K_tot_t = np.array(average_3(t_P_tot5))
    
    return t_P_tot5, K_Ex_5, K_Ey_5, K_tot, K_tot_t
    

def plot_kinetic_ene(part_id, savedir):
    width = 3.2 * 2
    height = width / 1.618 
    fig, ax = plt.subplots()

    ax.plot(K_tot_t,K_tot,color='green', label=r'$K_{tot}$', linewidth=2.0)
    ax.plot(t_P_tot5,K_Ex_5,color='red', linestyle='--',label=r'$K_{Ex}$',linewidth=2.0)
    ax.plot(t_P_tot5,K_Ey_5,color='blue',linestyle=':', label=r'$K_{Ey}$',linewidth=2.0)
    ax.plot(t_P_tot5,np.array(K_Ex_5)+np.array(K_Ey_5)+K_tot[0],color='cyan', linestyle='-.', linewidth=1.0, label=r'$K_{Ex}+K_{Ey}+K_{tot0}$',)


    ax.set_xlabel('time [ns]')
    ax.set_xlim([0,2.5])
    ax.set_ylabel('$K_{tot}$ [keV]')
#     ax.set_ylim([-40,120])

    ax.legend()
    ax.grid(linestyle='--', alpha=0.5)
    ax.set_title('ID = {:.0f}'.format(part_id))

    fig.set_size_inches(width, height)
    fig.savefig(savedir+'KE_ID_'+str(part_id)+'.png', bbox_inches='tight',dpi=300)
    plt.close()

# conversion constants for fields and momentum (when taken from TrackParticle)
kE = me*c*wr/qe  # V/m
kp = me*c        # kg m/s
kB = me*wr/qe    # T
conv_MeV_J = 10**6*qe

In [12]:
ekin0, num0, t_end0 = spect_load_1d(wkdir[0], 1.0, 2, 0.0, Lx)
# ekin1, num1, t_end1 = spect_load_1d(wkdir, 0.5, 1, 0.0, Lx)
# ekin2, num2, t_end2 = spect_load_1d(wkdir, 0.2, 1, 0.0, Lx)

In [13]:
width  = 3.2 * 2
height = width / 1.618
fig, ax = plt.subplots()
fig.subplots_adjust(left=.2, bottom=.2, right=.9, top=.9)
# ax.loglog(ekin2, num2, '-g', label='{:.1f} ns'.format(t_end2*1e9), lw=2.0)
# ax.loglog(ekin1, num1, '-b', label='{:.1f} ns'.format(t_end1*1e9), lw=2.0)
ax.loglog(ekin0, num0, '-r', label='{:.1f} ns'.format(t_end0*1e9), lw=2.0)
ax.tick_params(axis='both',which='both', direction='in')
ax.set_xlabel(r'$E_{p}$ (MeV)')
ax.set_ylabel(r'$E\frac{dN}{dE}\frac{1}{N}$')
# ax.set_xlim([0.009, 0.2])
ax.set_xticks([0.01, 0.05, 0.10])
ax.set_xticklabels([0.01, 0.05, 0.10])
# ax.set_ylim([1e-5, 1])
ax.legend(loc='best', numpoints=1, fancybox=True, fontsize=12)
ax.grid(which='both')
fig.set_size_inches(width, height)

In [15]:
# Fig.3, particle trajectories illustrating the acceleration mechanism

# prepare packages

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm, LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import LogLocator

# prepare to convert to center-of-mass reference frame 
n1   = 2e18     # drifting plasma, number density
n2   = 1e18     # ambient plasma, number density
v1   = 3.0e6    # drifting velocity, m/s
v2   = 0.0
vcom = (n1*v1 + n2*v2) / (n1+n2) # center-of-mass reference frame, m/s
vcom_mm_ns = vcom * 1e-6         # in unit of mm/ns

# prepare to convert to discontinuity reference frame
vcd =0 # 0.86*2+0.25 # mm/ns, read from the x-t plot.

# get data for 2D streak map
Bz_x_0, Bz_x, Bz_t, Bz = prepare_streak_2D(S, 0, 'Bz', vcom_mm_ns)
ni_x_0, ni_x, ni_t, ni = prepare_streak_2D(S, 0, 'Rho_ion1+Rho_ion2', vcom_mm_ns)

Bz_x_1, Bz_x_cd,  Bz_t_cd,  Bz_cd  = prepare_streak_2D(S, 0, 'Bz', vcd)
ni_x_1, ni_x_cd,  ni_t_cd,  ni_cd  = prepare_streak_2D(S, 0, 'Rho_ion1+Rho_ion2', vcd)
Ex_x_2, Ex_x_cd2, Ex_t_cd2, Ex_cd2 = prepare_streak_2D(S, 1, 'Ex', vcd)
Ey_x_2, Ey_x_cd2, Ey_t_cd2, Ey_cd2 = prepare_streak_2D(S, 1, 'Ey', vcd)
Bz_x_2, Bz_x_cd2, Bz_t_cd2, Bz_cd2 = prepare_streak_2D(S, 1, 'Bz', vcd)

In [ ]:
# particle selection condition
# TODO: in this Smilei formatted string, you can't use the self-defined constants.
condition_p1 = "any(t>0, ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 > 0.1) * ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 < 0.15))"
condition_p2 = "any(t>0, ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 > 0.1) * ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 > 0.15))"

# get particle tracer data

ion1t_x, ion1t_E, \
ion1t_px, ion1t_py, ion1t_pz, \
ion1t_time_ns, ion1t_x_sub, ion1t_E_sub, \
ion1t_Ex, ion1t_Ey, ion1t_Bz, \
ion1t_weight, ion1t_ts, ion1t_id = Particle_trajectory(S, 'ion1t', mp, condition_p1)

ion2t_x, ion2t_E, \
ion2t_px, ion2t_py, ion2t_pz, \
ion2t_time_ns, ion2t_x_sub, ion2t_E_sub, \
ion2t_Ex, ion2t_Ey, ion2t_Bz, \
ion2t_weight, ion2t_ts, ion2t_id = Particle_trajectory(S, 'ion2t', mp, condition_p2)

# ss_t2: 54 for ion1t, 46 for ion2t



In [9]:
ion2t_time_ns

array([0.00000000e+00, 1.77383418e-03, 3.54766836e-03, ...,
       2.65720360e+00, 2.65897744e+00, 2.66075127e+00])

In [10]:
## check every particle

for part_id in range(46):
    t_P_tot5, K_Ex_5, K_Ey_5, K_tot, K_tot_t = cal_kinetic_ene(part_id, ion2t_E, ion2t_Ex, 
                                                               ion2t_Ey, ion2t_px, ion2t_py, 
                                                               ion2t_time_ns)
    plot_kinetic_ene(part_id, wkdir[0])




In [13]:
for part_id in range(54):
    t_P_tot5, K_Ex_5, K_Ey_5, K_tot, K_tot_t = cal_kinetic_ene(part_id, ion1t_E, ion1t_Ex, 
                                                               ion1t_Ey, ion1t_px, ion1t_py, 
                                                               ion1t_time_ns)
    plot_kinetic_ene(part_id, wkdir[0])

In [14]:
nt = ion1t_time_ns.size
t_end = S.namelist.t_end / wr * 1e9
time_ns = np.linspace(0.,t_end,nt)

ion2t_x_frame = np.zeros_like(ion2t_x)
ion2t_x_vcd   = np.zeros_like(ion2t_x)
# ion2t_px_vcd  = zeros_like(ion2t_px)
# ion2t_E_vcd  = zeros_like(ion2t_E)
for i in range(46):     # Note that here the loop-number should be the num. of particles.
    ion2t_x_frame[:,i] = ion2t_x[:,i]*de*1e3 - vcom_mm_ns*time_ns - Lx/2.
    ion2t_x_vcd[:,i]   = ion2t_x[:,i]*de*1e3 - vcd       *time_ns - Lx/2.
#     ion2t_px_vcd[:,i]  = ion2t_px[:,i]*me*c - vcd*1e6*mp   # in the unit of kgm/s
#     ion2t_E_vcd[:,i]   = (ion2t_px_vcd[:,i]**2 + (ion2t_py[:,i]*me*c)**2 + (ion2t_pz[:,i]*me*c)**2) / 2. / mp / qe / 1e6
    
ion1t_x_frame = np.zeros_like(ion1t_x)
ion1t_x_vcd   = np.zeros_like(ion1t_x)
ion1t_px_vcd  = np.zeros_like(ion1t_px)
ion1t_E_vcd   = np.zeros_like(ion1t_E)
for i in range(111):
    ion1t_x_frame[:,i] = ion1t_x[:,i]*de*1e3 - vcom_mm_ns*time_ns - Lx/2.
    ion1t_x_vcd[:,i]   = ion1t_x[:,i]*de*1e3 - vcd       *time_ns - Lx/2.
#     ion1t_px_vcd[:,i]  = ion1t_px[:,i]*me*c - vcd*1e6*mp   # in the unit of kgm/s
#     ion1t_E_vcd[:,i]   = (ion1t_px_vcd[:,i]**2 + (ion1t_py[:,i]*me*c)**2 + (ion1t_pz[:,i]*me*c)**2) / 2. / mp / qe / 1e6

In [15]:
plt.plot(ion2t_x_vcd[:,3],  time_ns, lw=2.0, label='{:.1f}'.format(ion2t_id))
plt.plot(ion2t_x_vcd[:,2],  time_ns, lw=2.0, label='{:.1f}'.format(ion2t_id))
plt.plot(ion2t_x_vcd[:,1],  time_ns, lw=2.0, label='{:.1f}'.format(ion2t_id))
plt.plot(ion1t_x_vcd[:,1],  time_ns, lw=2.0)
plt.legend()

In [21]:
# From this point, the post-process will become quite customized.
# It will be highly dependent on what you want to show.

# Draw the figure of x-t-ni/Bz
width = 3.2 * 2
height = width / 1.618
fig, axs = plt.subplots()

# cs1 = axs.pcolormesh(Bz_x_cd2, Bz_t_cd2, np.log10(Bz_cd2**2),cmap=plt.cm.Greys)
# cs1 = axs.pcolormesh(ni_x, ni_t, ni,cmap=plt.cm.jet)#,vmin=0,vmax=6.0e18)
cs1 = axs.pcolormesh(ni_x_cd, ni_t_cd, ni_cd,cmap=plt.cm.jet,vmin=0,vmax=16.0)
cs1.cmap.set_under(color='white')
cs1.cmap.set_over(color='black')
ax1_divider = make_axes_locatable(axs)
cax1=ax1_divider.append_axes("right", size="4%", pad="2%")
# cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$log_{10}(|B_z|^2)$')
cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$n_i$ (1e18 cm$^{-3}$)')
cbar1.ax.tick_params()
cax1.xaxis.set_ticks_position("top")
axs.tick_params(axis='both')
axs.set_xlabel('x (mm)')
axs.set_ylabel('t (ns)')

axs.plot(ion2t_x_vcd[:,0:46:4],  time_ns, lw=2.0, ls='--')
axs.plot(ion1t_x_vcd[:,0:111:10],  time_ns, lw=2.0)

# # axs.plot(ion2t_x_frame[:,0],  time_ns, lw=2.0, color='blue',   ls='--')
# # axs.plot(ion2t_x_frame[:,7],  time_ns, lw=2.0, color='red',  ls='--')
# # axs.plot(ion2t_x_frame[:,14], time_ns, lw=2.0, color='green', ls='--')
# # axs.plot(ion1t_x_frame[:,0],  time_ns, lw=2.0, color='black')
# # axs.plot(ion1t_x_frame[:,2],  time_ns, lw=2.0, color='m')

# # axs.plot(ion2t_x_vcd[:,0],  time_ns, lw=2.0, color='blue',   ls='--')
# axs.plot(ion2t_x_vcd[:,7],  time_ns, lw=2.0, color='red',  ls='--')
# # axs.plot(ion2t_x_vcd[:,14], time_ns, lw=2.0, color='green', ls='--')
# axs.plot(ion1t_x_vcd[:,0],  time_ns, lw=2.0, color='black')
# axs.plot(ion1t_x_vcd[:,2],  time_ns, lw=2.0, color='m')


axs.set_xlim([-2,1.0])
axs.set_ylim([0,2.0])
axs.set_aspect('equal')

fig.set_size_inches(width,height)
fig.tight_layout()



In [22]:
fig.savefig('/Users/yao/Desktop/'+'trace_ni_xt2'+'.png',bbox_inches='tight',dpi=600)

In [74]:
width = 3.2 * 2
height = width / 1.618
fig, axs = plt.subplots()
part_id = 3
# axs.plot(ion2t_x_frame[:,0],  ion2t_E[:,0],  lw=2.0, ls='--', color='blue')
# axs.plot(ion2t_x_frame[:,7],  ion2t_E[:,7],  lw=2.0, ls='--', color='red')
# axs.plot(ion2t_x_frame[:,14], ion2t_E[:,14], lw=2.0, ls='--', color='green')
# axs.plot(ion1t_x_frame[:,0],  ion1t_E[:,0],  lw=2.0, color='black')
# axs.plot(ion1t_x_frame[:,2],  ion1t_E[:,2],  lw=2.0, color='m')

# axs.plot(ion2t_x_vcd[:,0],  ion2t_E[:,0],  lw=2.0, color='blue',   ls='--')
# axs.plot(ion2t_x_vcd[:,7],  ion2t_E[:,7],  lw=2.0, color='red',  ls='--')
# axs.plot(ion2t_x_vcd[:,14], ion2t_E[:,14], lw=2.0, color='green', ls='--')
# axs.plot(ion1t_x_vcd[:,0],  ion1t_E[:,0],  lw=2.0, color='black')
# axs.plot(ion1t_x_vcd[:,3],  ion1t_E[:,3],  lw=2.0, color='m')

# axs.plot(ion2t_x_vcd[:,0],  ion2t_E[:,0],  lw=2.0, color='blue',   ls='--')
# axs.plot(ion2t_x_vcd[:,7],  ion2t_E[:,7],  lw=2.0, color='red',  ls='--')
# axs.plot(ion2t_x_vcd[:,14], ion2t_E[:,14], lw=2.0, color='green', ls='--')
axs.plot(ion1t_x[:,part_id],  ion1t_px[:,part_id]*(me*c)/mp/1e3,  lw=2.0, color='blue', label=r'$V_x$')
axs.plot(ion1t_x[:,part_id],  ion1t_py[:,part_id]*(me*c)/mp/1e3,  lw=2.0, color='red', label=r'$V_y$')
# axs.plot(ion1t_x_vcd[:,3],  ion1t_E[:,3],  lw=2.0, color='m')

# axs.set_xlim([-2,2])
# axs.set_ylim([0,0.08])
axs.legend()
axs.set_xlabel('x [mm]')
axs.set_ylabel('v [km/s]')

Text(0, 0.5, 'v [km/s]')

In [24]:
width = 3.2 * 2
height = width 
fig, axs = plt.subplots()

id2 = 0

# axs.plot(ion2t_px_vcd[:,0],  ion2t_py[:,0],  lw=2.0, ls='--', color='blue')
axs.plot(ion2t_px[:1400,id2]*(me*c)/mp/1e3,  ion2t_py[:1400,id2]*(me*c)/mp/1e3,  lw=2.0, ls='--', color='red')
# axs.plot(ion2t_px[:,14], ion2t_py[:,14], lw=2.0, ls='--', color='green')
axs.plot(ion1t_px[:,0]*(me*c)/mp/1e3,  ion1t_py[:,0]*(me*c)/mp/1e3,  lw=2.0, color='black')
axs.plot(ion1t_px[:,40]*(me*c)/mp/1e3,  ion1t_py[:,40]*(me*c)/mp/1e3,  lw=2.0, color='m')

# axs.plot(ion2t_x_vcd[:,0],  ion2t_px[:,0],  lw=2.0, color='blue',   ls='--')
# axs.plot(ion2t_x_vcd[:,7],  ion2t_px[:,7],  lw=2.0, color='red',  ls='--')
# axs.plot(ion2t_x_vcd[:,14], ion2t_px[:,14], lw=2.0, color='green', ls='--')
# axs.plot(ion1t_x_vcd[:,0],  ion1t_px[:,0],  lw=2.0, color='black')
# # axs.plot(ion1t_x_vcd[:,3],  ion1t_px[:,3],  lw=2.0, color='m')
# axs.plot(ion1t_x_vcd[:,0],  ion1t_py[:,0],  lw=2.0, color='black', ls='--')

# axs.plot(time_ns,  ion1t_px[:,0],  lw=2.0, color='black')
# # axs.plot(ion1t_x_vcd[:,3],  ion1t_px[:,3],  lw=2.0, color='m')
# axs.plot(time_ns,  ion1t_py[:,0],  lw=2.0, color='black', ls='--')

# axs.plot(time_ns,  ion2t_px[:,0],  lw=2.0, color='blue',   ls='-')
# axs.plot(time_ns,  ion2t_py[:,0],  lw=2.0, color='blue',   ls='--')

# axs.plot(ion1t_px[:,0],  ion1t_py[:,0],  lw=2.0, color='black')


axs.set_xlim([-4000,6000])
axs.set_ylim([-5000,5000])
axs.grid()

axs.set_xlabel(r'$V_x (km/s)$')
axs.set_ylabel(r'$V_y (km/s)$')
axs.set_aspect('equal')
fig.tight_layout()

fig.set_size_inches(width,height)

In [25]:
fig.savefig('/Users/yao/Desktop/'+'trace_vxvy2'+'.png',bbox_inches='tight',dpi=600)

In [ ]:
# Draw the figure of x-t-ni-E

width = 3.2 * 3
height = width / 1.618 
fig, axs = plt.subplots()

cs1 = axs.pcolormesh(Bz_x_cd2, Bz_t_cd2, np.log10(Bz_cd2**2),cmap=plt.cm.Greys)
# cs1 = axs.pcolormesh(ni_x, ni_t, ni,cmap=plt.cm.Greys,vmin=0,vmax=6.0e18)
# cs1 = axs.pcolormesh(ni_x_cd, ni_t_cd, ni_cd,cmap=plt.cm.Greys,vmin=0,vmax=16)
cs1.cmap.set_under(color='white')
cs1.cmap.set_over(color='black')
ax1_divider = make_axes_locatable(axs)
cax1=ax1_divider.append_axes("right", size="4%", pad="2%")
cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$log_{10}(|B_z|^2)$')
# cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$n_i \ [1e18 \ cm^{-3}]$ ')
cbar1.ax.tick_params()
cax1.xaxis.set_ticks_position("top")
axs.tick_params(axis='both')
axs.set_xlabel('x (mm)',fontsize=18)
axs.set_ylabel('t (ns)',fontsize=18)

axs.set_xlim([-2,1])
axs.set_ylim([0,2])


id = 20
# points = np.array([ion2t_x_frame[:,0], time_ns]).T.reshape(-1, 1, 2)
points = np.array([ion2t_x_vcd[:,id], time_ns]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = plt.Normalize(ion2t_E[:,id].min(), ion2t_E[:,id].max())
lc = LineCollection(segments, norm=norm4,cmap=plt.cm.jet)
# Set the values used for colormapping
lc.set_array(ion2t_E[:,id])
lc.set_linewidth(3)
line = axs.add_collection(lc)

id = 50#97
points2 = np.array([ion1t_x_vcd[:,id], time_ns]).T.reshape(-1, 1, 2)
segments2 = np.concatenate([points2[:-1], points2[1:]], axis=1)

norm2 = plt.Normalize(ion1t_E[:,id].min(), ion1t_E[:,id].max())
lc2 = LineCollection(segments2, norm=norm4,cmap=plt.cm.jet)
# Set the values used for colormapping
lc2.set_array(ion1t_E[:,id])
lc2.set_linewidth(3)
line2 = axs.add_collection(lc2)

id = 30
points3 = np.array([ion1t_x_vcd[:,id], time_ns]).T.reshape(-1, 1, 2)
segments3 = np.concatenate([points3[:-1], points3[1:]], axis=1)

norm3 = plt.Normalize(ion1t_E[:,id].min(), ion1t_E[:,id].max())
lc3 = LineCollection(segments3, norm=norm4,cmap=plt.cm.jet)
# Set the values used for colormapping
lc3.set_array(ion1t_E[:,id])
lc3.set_linewidth(3)
line3 = axs.add_collection(lc3)

id = 10
points4= np.array([ion1t_x_vcd[:,id], time_ns]).T.reshape(-1, 1, 2)
segments4 = np.concatenate([points4[:-1], points4[1:]], axis=1)

norm4 = plt.Normalize(ion1t_E[:,id].min(), ion1t_E[:,id].max())
lc4 = LineCollection(segments4, norm=norm4,cmap=plt.cm.jet)
# Set the values used for colormapping
lc4.set_array(ion1t_E[:,id])
lc4.set_linewidth(3)
line4 = axs.add_collection(lc4)

cax0=fig.add_axes([0.12,0.93,0.72,0.02])
cbar0=fig.colorbar(lc4, ax=fig, cax=cax0, orientation="horizontal")
cbar0.ax.tick_params()
cax0.xaxis.set_ticks_position("bottom")
cbar0.ax.set_title('E (MeV)',fontsize=18)

# fig.tight_layout()
fig.set_size_inches(width, height)

# fig.savefig('/Users/yz/Desktop/'+'trace_latest1'+'.png',bbox_inches='tight',dpi=300)

In [49]:
fig.savefig('/Users/yao/Desktop/'+'trace_latest1'+'.png',bbox_inches='tight',dpi=600)

In [ ]:
for i in range(111):
    print(i, np.max(ion1t_E[:,i]))

In [51]:
# Draw the figure of x-t-ni-Fx

# prepare Lorentz force
# Here, the Ex contribution is not small, but high-frequency.
# fx = qe * (ion2t_Ex[:,0]*me*wr*c/qe + ion2t_py[:,0]*me*c/mp * ion2t_Bz[:,0]*me*wr/qe)
fx = qe * (ion2t_py[:,0]*me*c/mp * ion2t_Bz[:,0]*me*wr/qe)
fy = qe * (ion2t_px[:,0]*me*c/mp * ion2t_Bz[:,0]*me*wr/qe) * -1.0

# Prepare the work done by Lorentz force
xx  = ion2t_x[:,0] * de
# ddx = zeros_like(xx)
# wx  = zeros_like(xx)
# for i in range(xx.size-1):
#     ddx[i+1] = xx[i+1] - xx[i]
#     wx[i+1]  = wx[i] + fx[i+1] * ddx[i+1]
wx = fx * xx
wx_norm = wx / wx.max()
wx_mev = wx / qe / 1e6  # convert it into MeV
# dde = zeros_like(ion2t_E[:,0])
# for i in range(dde.size-1):
#     dde[i+1] = ion2t_E[i+1,0] - ion2t_E[i,0]

width = 3.2 * 3
height = width / 1.618 / 2 
fig, axs = plt.subplots()

# cs1 = axs.pcolormesh(Bz_x, Bz_t, np.log10(Bz**2),cmap=plt.cm.Greys)
# cs1 = axs.pcolormesh(ni_x, ni_t, ni,cmap=plt.cm.Greys,vmin=0,vmax=6.0e18)
cs1 = axs.pcolormesh(ni_x_cd, ni_t_cd, ni_cd,cmap=plt.cm.Greys,vmin=0,vmax=16)
# cs1 = axs.pcolormesh(ni_x_0, ni_t_0, ni_0,cmap=plt.cm.Greys,vmin=0,vmax=6.0e18)
cs1.cmap.set_under(color='white')
cs1.cmap.set_over(color='black')
ax1_divider = make_axes_locatable(axs)
cax1=ax1_divider.append_axes("right", size="4%", pad="2%")
# cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$log_{10}(|B_z|^2)$')
cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$n_i\ (1e18\ cm^{-3})$ ')
cbar1.ax.tick_params()
cax1.xaxis.set_ticks_position("top")
axs.tick_params(axis='both')
axs.set_xlabel('x (mm)')
axs.set_ylabel('t (ns)')

axs.set_xlim([-2,1])
axs.set_ylim([0,2])

# points = np.array([ion2t_x_frame[:,0], time_ns]).T.reshape(-1, 1, 2)
points = np.array([ion2t_x_vcd[:,0], time_ns]).T.reshape(-1, 1, 2)
# points = np.array([ion2t_x[:,0]*de*1e3, time_ns]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = plt.Normalize(-1.0, 1.0)
# norm = plt.Normalize(ion2t_E[:,0].min(), ion2t_E[:,0].max())
lc = LineCollection(segments, norm=norm,cmap=plt.cm.seismic)
# lc = LineCollection(segments, norm=norm,cmap=newcmap)
# Set the values used for colormapping
# lc.set_array(dde)
lc.set_array(wx_norm)
lc.set_linewidth(5)
line = axs.add_collection(lc)

cax0=fig.add_axes([0.12,0.93,0.72,0.02])
cbar0=fig.colorbar(lc, ax=fig, cax=cax0, orientation="horizontal")
cbar0.ax.tick_params()
cax0.xaxis.set_ticks_position("bottom")
cbar0.ax.set_title('Wx (A.U.)',fontsize=18)
# cbar0.ax.set_title('Fx [kgm/s2]',fontsize=18)
# fig.set_size_inches(width, height)

Text(0.5, 1.0, 'Wx (A.U.)')

In [59]:
plt.plot(ni_cd[100,:])

In [52]:
ion1t_1d = 50

ion1t_Fx  = ion1t_Ex[:,ion1t_id] + ion1t_py[:,ion1t_id] * ion1t_Bz[:,ion1t_id]
ion1t_Fx1 = ion1t_Ex[:,ion1t_id] 
ion1t_Fx2 = ion1t_py[:,ion1t_id] * ion1t_Bz[:,ion1t_id]

ion1t_Fy  = ion1t_Ey[:,ion1t_id] - ion1t_px[:,ion1t_id] * ion1t_Bz[:,ion1t_id]
ion1t_Fy1 = ion1t_Ey[:,ion1t_id] 
ion1t_Fy2 = -1.0 * ion1t_px[:,ion1t_id] * ion1t_Bz[:,ion1t_id]

ion2t_Fx  = ion2t_Ex[:,ion2t_id] + ion2t_py[:,ion2t_id] * ion2t_Bz[:,ion2t_id]
ion2t_Fx1 = ion2t_Ex[:,ion2t_id] 
ion2t_Fx2 = ion2t_py[:,ion2t_id] * ion2t_Bz[:,ion2t_id]

ion2t_Fy  = ion2t_Ey[:,ion2t_id] - ion2t_px[:,ion2t_id] * ion2t_Bz[:,ion2t_id]
ion2t_Fy1 = ion2t_Ey[:,ion2t_id] 
ion2t_Fy2 = -1.0 * ion2t_px[:,ion2t_id] * ion2t_Bz[:,ion2t_id]

In [53]:
fig, ax = plt.subplots()

ax.plot(ion1t_x[:,ion1t_id]*de*1e3, ion1t_Fx/(me*c*wr)/1e8, color='red', lw=2.0, label='Fx')
ax.plot(ion1t_x[:,ion1t_id]*de*1e3, ion1t_Fy/(me*c*wr)/1e8, color='blue', lw=2.0, label='Fy')
ax.plot(ion1t_x[:,ion1t_id]*de*1e3, ion1t_time_ns/wr*1e9, color='black', lw=2.0, label='trajectory')
ax.set_xlabel('position x (mm)')
ax.set_ylabel('Lorentz force (normed)')
ax.legend()
ax.grid()
fig.tight_layout()
plt.show()

In [56]:
ion1t_Fx.size

1501

In [58]:
ion1t_W  = np.zeros_like(ion1t_Fx)

for i in range(ion1t_Fx.size-1):
    ion1t_W[i+1]  = ion1t_W[i]  + (ion1t_Fx[i+1]+ion1t_Fx[i])/2.  * (ion1t_x[i+1,ion1t_id] - ion1t_x[i,ion1t_id])*1e-3/de
#     ion1t_W[i+1]  = ion1t_W[i+1]  + ion1t_Fx[i+1]  * (ion1t_x[i+1,ion1t_id] - ion1t_x[i,ion1t_id])*1e-3/de


ion2t_W  = np.zeros_like(ion2t_Fx)

for i in range(ion1t_Fx.size-1):
    ion2t_W[i+1]  = ion2t_W[i]  + (ion2t_Fx[i+1]+ion2t_Fx[i])/2.  * (ion2t_x[i+1,ion2t_id] - ion2t_x[i,ion2t_id])*1e-3/de



In [59]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('position x (mm)')
# ax1.set_xlabel('time (ns)')
ax1.set_ylabel('Work done by Fx', color=color)
ax1.set_ylim(ion1t_W1.min(),-ion1t_W1.min())
ax1.plot(ion1t_x[:,ion1t_id], ion1t_W1, color=color, lw=2.0, label='All')
# ax1.plot(ion1t_x[:,ion1t_id], ion1t_Fx1, color=color, lw=2.0, ls='--', label='Ex')
# ax1.plot(ion1t_x[:,ion1t_id], ion1t_Fx2, color=color, lw=2.0, ls='-.', label='vyBz')
# ax1.plot(ion1t_time_ns, ion1t_E_sub, color=color)
ax1.tick_params(axis='y',labelcolor=color)
ax1.grid()
ax2 = ax1.twinx()


dEx = ion1t_px[:,ion1t_id]**2 / 2.0 / mp / qe / 1e6
dEy = ion1t_py[:,ion1t_id]**2 / 2.0 / mp / qe / 1e6
dEz = ion1t_pz[:,ion1t_id]**2 / 2.0 / mp / qe / 1e6
color= 'tab:blue'
ax2.set_ylabel(r'$\delta E$', color=color)
ax2.set_ylim(0,dEx.max())
ax2.plot(ion1t_x[:,ion1t_id], dEx, color=color, lw=2.0, ls='-',  label=r'$\delta E_x$')
ax2.plot(ion1t_x[:,ion1t_id], dEy, color='black', lw=2.0, ls='-',  label=r'$\delta E_y$')
# ax2.plot(ion1t_x[:,ion1t_id], dEz, color='green', lw=2.0, ls='-',  label=r'$\delta E_z$')
# ax1.plot(ion1t_x[:,ion1t_id], ion1t_Fy1, color=color, lw=2.0, ls='--', label='Ey')
# ax1.plot(ion1t_x[:,ion1t_id], ion1t_Fy2, color=color, lw=2.0, ls='-.', label='vxBz')
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend()
fig.tight_layout()
plt.show()


# # plt.plot(ion1t_x[:,ion1t_id], ion1t_W, '-k', lw=2.0, label='Work done by Fx')
# plt.plot(ion1t_time_ns,ion1t_W)
# plt.plot(ion1t_time_ns,ion1t_E_sub)
# plt.xlabel('position x (mm)')
# plt.ylabel('Work done by Lorentz force (Arb. Unit)')
# plt.legend()

NameError: name 'ion1t_W1' is not defined

In [8]:
# ion x-px
xpx0 = S0.ParticleBinning(0,units=['mm', 'ns', '1000 km/s'],
                  data_log=True,
                   # xmin=5,
                   # xmax=11,
                   # ymin=-3,
                   # ymax=6,
                  # vmin=-1,
                  # vmax=4,
                  cmap=newcmap
                  # ).animate(movie='/Users/yao/Desktop/xpx_i_t1.gif')
                  )#.slide()

xpx1 = S1.ParticleBinning(0,units=['mm', 'ns', '1000 km/s'],
                  data_log=True,
                   # xmin=5,
                   # xmax=11,
                   # ymin=-3,
                   # ymax=6,
                  # vmin=-1,
                  # vmax=4,
                  cmap=newcmap
                  # ).animate(movie='/Users/yao/Desktop/xpx_i_t1.gif')
                  )#.slide()

happi.multiSlide(xpx0, xpx1, shape=[2,1])

NameError: name 'S0' is not defined

In [8]:
# ion x
S.ParticleBinning(0,units=['mm', 'ns', 'km/s','cm^-3'],
                  # data_log=True,
                   # sum={'vx':'all'},
                   # xmin=5,
                   # xmax=11,
                  # vmin=-2,
                  # vmax=4,
                  # cmap=newcmap
                  # ).animate(movie='/Users/yao/Desktop/x_i_t1.gif')
                  ).slide()


#0 - Number density of species # 0,4,8
    x from 0 to 6144 in 500 steps 
    vx from -0.05 to 0.05 in 500 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size



In [10]:
# eon x-px
S.ParticleBinning(3,units=['mm', 'ns', 'km/s'],
                  data_log=True,
                  vmin=-2,
                  vmax=4,
                  cmap=newcmap
                  ).slide()


#3 - Number density of species # 4
    x from 0 to 8192 in 200 steps 
    vx from -0.05 to 0.05 in 200 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size



In [11]:
# ion spectrum
S.ParticleBinning(5,units=['mm', 'ns', 'km/s','MeV'],
                  data_log=True,
                  # vmin=-2,
                  # vmax=4,
                   sum={'x':'all'},
                   # ymin=0,
                   # ymax=200,
                  cmap=newcmap
                  ).slide()


#5 - Number density of species # 4
    x from 0 to 8192 in 200 steps 
    ekin from 0 to 1 in 200 steps 

average for all x
The value in each bin is the sum of the `deposited_quantity` divided by the bin size



In [16]:
# eon spectrum
S.ParticleBinning(3,units=['mm', 'ns', 'km/s','keV'],
                  data_log=True,
                  # vmin=-2,
                  # vmax=4,
                   sum={'x':'all'},
                   ymin=0,
                   ymax=200,
                  cmap=newcmap
                  ).slide()


#3 - Number density of species # 4
    x from 0 to 8192 in 200 steps 
    vx from -0.05 to 0.05 in 200 steps 

average for all x
The value in each bin is the sum of the `deposited_quantity` divided by the bin size



In [22]:
# ion vx-vy
S.ParticleBinning(4,units=['mm', 'ns', 'km/s'],
                  data_log=True,
                  vmin=-2,
                  vmax=6,
                  cmap=newcmap,
                   aspect='equal',
                  ).slide()


#4 - Number density of species # 0
    x from 0 to 8192 in 200 steps 
    ekin from 0 to 1 in 200 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size



In [24]:
# eon vx-vy
S.ParticleBinning(1,units=['mm', 'ns', 'km/s'],
                  data_log=True,
                  vmin=-2,
                  vmax=6,
                  cmap=newcmap,
                   aspect='equal',
                  ).slide()


#1 - Number density of species # 0,4
    x from 0 to 8192 in 200 steps 
    ekin from 0 to 1 in 200 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size



In [10]:
Bz = S.Field(0,'Bz',units=['mm','ns'],
         label='Bz',
              color='blue',
              linestyle='--',
             )
        # ).slide()

ni = S.ParticleBinning("#0*0.03",units=['mm', 'ns'],
                   sum={'vx':'all'},
                        label='ni',
                        color='red',
                       )
                  # ).slide()

happi.multiSlide(Bz, ni)

In [8]:
Ex0 = S.Field(0,'Ex',units=['mm','ns','V/m'],
         label='Ex0',
              color='blue',
              linestyle='--',
             )

Ex1 = S.Field(1,'Ex',units=['mm','ns','V/m'],
         label='Ex0',
              color='red',
              linestyle='-',
             )

happi.multiSlide(Ex0,Ex1,
                 # shape=[1,3]
                )

In [21]:
S.Field(0,'(Rho_ion1+Rho_ion2)',units=['mm','ns','T','cm^-3'],
         label='ion',
        ).slide()

Field diagnostic #0: (Rho_ion1+Rho_ion2)


In [19]:
S.Field(0,'Bz',units=['mm','ns','T'],
         label='Bz',
         # xmin=5,
                   # xmax=11,
              # color='blue',
              # linestyle='--',
        ).streak()
             # ).animate(movie='/Users/yao/Desktop/bz_t1.gif')

Field diagnostic #0: Bz


In [20]:
S.ParticleBinning("(#6+#8)/(#7+#9)", 
                   units=['mm','ns','cm^-3','km/s','eV'],
                   # xmin=5,
                   # xmax=11,
                  ).slide()



#6 - Pressure yy of species # 2
    x from 0 to 8192 in 1500 steps 

#7 - Number density of species # 2
    x from 0 to 8192 in 1500 steps 

#8 - Pressure yy of species # 6
    x from 0 to 8192 in 1500 steps 

#9 - Number density of species # 6
    x from 0 to 8192 in 1500 steps 

Operation : (#6+#8)/(#7+#9)
The value in each bin is the sum of the `deposited_quantity` divided by the bin size



/opt/homebrew/lib/python3.9/site-packages/pint/facets/plain/quantity.py:986: RuntimeWarning: invalid value encountered in divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [11]:
S.ParticleBinning()

Exception: `diagNumber` is not defined
Printing available ParticleBinning:
------------------------------------------------
#0 - Number density of species # 0,4
    x from 0 to 4096 in 200 steps 
    vx from -0.05 to 0.05 in 200 steps 

#1 - Number density of species # 0,4
    x from 0 to 4096 in 200 steps 
    ekin from 0 to 1 in 200 steps 

#2 - Number density of species # 0
    x from 0 to 4096 in 200 steps 
    vx from -0.05 to 0.05 in 200 steps 

#3 - Number density of species # 4
    x from 0 to 4096 in 200 steps 
    vx from -0.05 to 0.05 in 200 steps 

#4 - Number density of species # 0
    x from 0 to 4096 in 200 steps 
    ekin from 0 to 1 in 200 steps 

#5 - Number density of species # 4
    x from 0 to 4096 in 200 steps 
    ekin from 0 to 1 in 200 steps 

#6 - Pressure yy of species # 2
    x from 0 to 4096 in 1500 steps 

#7 - Number density of species # 2
    x from 0 to 4096 in 1500 steps 

#8 - Pressure yy of species # 6
    x from 0 to 4096 in 1500 steps 

#9 - Number

In [21]:
S.ParticleBinning("(#10+#12)/(#11+#13)", 
                   units=['mm','ns','cm^-3','km/s','eV'],
                   # xmin=5,
                   # xmax=11,                   
                  ).slide()



#10 - Pressure yy of species # 0
    x from 0 to 8192 in 1500 steps 

#11 - Number density of species # 0
    x from 0 to 8192 in 1500 steps 

#12 - Pressure yy of species # 4
    x from 0 to 8192 in 1500 steps 

#13 - Number density of species # 4
    x from 0 to 8192 in 1500 steps 

Operation : (#10+#12)/(#11+#13)
The value in each bin is the sum of the `deposited_quantity` divided by the bin size



In [25]:
S.ParticleBinning(14, 
                   units=['mm','ns','cm^-3','1000 km/s','keV'],
                   data_log=True,
                   cmap=newcmap,
                   vmin=1,
                   vmax=5,
                   xmin=-6,
                   xmax=6,
                   ymin=-6,
                   ymax=6,
                   aspect=1
                  ).slide()


#14 - Number density of species # 0,4
    vx from -0.05 to 0.05 in 200 steps 
    vy from -0.05 to 0.05 in 200 steps 

Operation : #14
The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0]



In [11]:
# ion x-px
S.ParticleBinning(0,units=['mm', 'ns', '1000 km/s'],
                  data_log=True,
                   # xmin=5,
                   # xmax=11,
                   # ymin=-1,
                   # ymax=5,
                  # vmin=-1,
                  # vmax=4,
                  cmap=newcmap,
                   # aspect=1
                  # ).animate(movie='/Users/yao/Desktop/xpx_i_t1.gif')
                  ).slide()


#0 - Number density of species # 0,4,8
    x from 0 to 6144 in 500 steps 
    vx from -0.05 to 0.05 in 500 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size



In [1]:
import numpy as np
import matplotlib as mpl
# mpl.use('pdf')
import matplotlib.pyplot as plt

In [3]:
plt.rc('font', family='sans-serif', serif='Arial')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

In [45]:
time = np.array([2, 3, 4, 6, 8, 10, 12]) # ns
size_on_screen = np.array([2.1, 2.69, 3, 5.37, 6.24, 7.15, 8.26]) # cm

def rescale(x):
    return x*17/6.79

In [46]:
size_in_physics = rescale(size_on_screen) # in mm
print(size_in_physics)

[ 5.25773196  6.73490427  7.51104566 13.44477172 15.62297496 17.90132548
 20.68041237]


In [34]:
def lin_reg(x_data, y_data):
    n = np.size(x_data)
    
    xm = np.mean(x_data)
    ym = np.mean(y_data)
    S1 = np.sum(x_data*y_data) - n*xm*ym
    S2 = np.sum(x_data**2) - n*xm**2
    
    a1 = S1/S2
    a0 = ym - a1*xm
    
    return (a0,a1)

In [47]:
a0_7bar, a1_7bar = lin_reg(time, size_in_physics/2.)

In [56]:
width  = 3.487 
height = width / 1.618 * 1.5

fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)


x = np.linspace(time[0],time[-1],50)

ax.plot(x, a0_7bar+a1_7bar*x, 'b-',
        label='Expansion velocity v0 = {:.1e} km/s'.format(a1_7bar*1e3)
       )

ax.plot(time, size_in_physics/2., 'ro',
        label='Experimental data',
       )



ax.set_xlabel("Time (ns)")
ax.set_ylabel("Radius (mm)")
ax.set_xlim([time[0],time[-1]])
ax.set_ylim([2,12])
# ax.set_title('Expansion velocity v = {:.1e} m/s'.format(a1*1e3))
ax.legend()

fig.set_size_inches(width, height)



# fig.savefig('/Users/yao/Desktop/plot.pdf')

In [57]:
fig.savefig('/Users/yao/Desktop/plot_7bars.pdf')

In [58]:
# for pressure of 4 bars H2
x_data = np.array([3.0,   5.0,   6.0,   7.0,   8.0])
y_data = np.array([9.51, 13.97, 15.45, 18.38, 20.68])
a0,a1 = lin_reg(x_data,y_data/2.)

In [59]:
width  = 3.487 
height = width / 1.618 * 1.5

fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)


x = np.linspace(2,10,50)

ax.plot(x, a0+a1*x, 'b-',
        label='Expansion velocity v0 = {:.1e} km/s'.format(a1*1e3)
       )

ax.plot(x_data,y_data/2.,'ro',
        label='Experimental data',
       )



ax.set_xlabel("Time (ns)")
ax.set_ylabel("Radius (mm)")
ax.set_xlim([3,8])
ax.set_ylim([4,12])
# ax.set_title('Expansion velocity v = {:.1e} m/s'.format(a1*1e3))
ax.legend()

fig.set_size_inches(width, height)



# fig.savefig('/Users/yao/Desktop/plot.pdf')

In [60]:
fig.savefig('/Users/yao/Desktop/plot.pdf')

In [11]:
a0

2.729054054054103

In [35]:
# for pressure of 4 bars H2
x_data = np.array([3.0,   5.0,   6.0,   7.0,   8.0]) # time in ns
y_data = np.array([9.51, 13.97, 15.45, 18.38, 20.68]) # size (diameter) in mm

In [6]:
0.015 * 3e8 /3

1500000.0

In [8]:
4.5e-3 / 3e-9 * 1e-9 * 1e3

1.5

In [33]:
condition_p1 = "any(t>0, ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 > 0.1) * ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 < 0.15))"


0.006666666666666667

In [ ]:
(px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 > 0.1

In [1]:
(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6

0.0001393995098039216

In [2]:
0.1 / ((9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6)

717.3626373626372

In [3]:
import numpy as np

In [5]:
np.sqrt(1. / ((9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6))

84.69726308226478

In [6]:
me = 9.1e-31
c = 3e8

In [7]:
me * c

2.73e-22